In [1]:
from collections import deque

class Module:
    def __init__(self, name, type, outputs):
        self.name = name
        self.type = type
        self.outputs = outputs

        if type == "%":
            self.memory = "off"
        else:
            self.memory = {}

    def __repr__(self):
        return self.name + "{type=" + self.type + ",outputs=" + ",".join(self.outputs) + ",memory=" + str(self.memory) + "}"

modules = {}
broadcast_targets = []

with open('/content/input 20.txt', 'r') as file:
    for line in file:
        left, right = line.strip().split(" -> ")
        outputs = right.split(", ")
        if left == "broadcaster":
            broadcast_targets = outputs
        else:
            type = left[0]
            name = left[1:]
            modules[name] = Module(name, type, outputs)

for name, module in modules.items():
    for output in module.outputs:
        if output in modules and modules[output].type == "&":
            modules[output].memory[name] = "lo"

lo = hi = 0

for _ in range(1000):
    lo += 1
    q = deque([("broadcaster", x, "lo") for x in broadcast_targets])

    while q:
        origin, target, pulse = q.popleft()

        if pulse == "lo":
            lo += 1
        else:
            hi += 1

        if target not in modules:
            continue

        module = modules[target]

        if module.type == "%":
            if pulse == "lo":
                module.memory = "on" if module.memory == "off" else "off"
                outgoing = "hi" if module.memory == "on" else "lo"
                for x in module.outputs:
                    q.append((module.name, x, outgoing))
        else:
            module.memory[origin] = pulse
            outgoing = "lo" if all(x == "hi" for x in module.memory.values()) else "hi"
            for x in module.outputs:
                q.append((module.name, x, outgoing))

print(lo * hi)


681194780


In [1]:
import re
from copy import deepcopy
from math import gcd
from collections import defaultdict, Counter, deque
import heapq
import math

with open("/content/input 20.txt", "r") as file:
    D = file.read().strip()
    L = D.split('\n')

G = [[c for c in row] for row in L]
R = len(G)
C = len(G[0])

def lcm(xs):
    ans = 1
    for x in xs:
        ans = (ans * x) // math.gcd(x, ans)
    return ans

TYP = {}

R = {}
for line in L:
    src, dest = line.split('->')
    src = src.strip()
    dest = dest.strip()
    dest = dest.split(', ')
    R[src] = dest
    TYP[src[1:]] = src[0]

def adjust(y):
    if y in TYP:
        return TYP[y] + y
    else:
        return y

FROM = {}
INV = defaultdict(list)
for x, ys in R.items():
    R[x] = [adjust(y) for y in ys]
    for y in R[x]:
        if y[0] == '&':
            if y not in FROM:
                FROM[y] = {}
            FROM[y][x] = 'lo'
        INV[y].append(x)

assert len(INV['rx']) == 1
assert INV['rx'][0][0] == '&'
WATCH = INV[INV['rx'][0]]

lo = 0
hi = 0
Q = deque()
ON = set()
PREV = {}
COUNT = defaultdict(int)
TO_LCM = []
for t in range(1, 10**8):
    Q.append(('broadcaster', 'button', 'lo'))

    while Q:
        x, from_, typ = Q.popleft()

        if typ == 'lo':
            if x in PREV and COUNT[x] == 2 and x in WATCH:
                TO_LCM.append(t - PREV[x])
            PREV[x] = t
            COUNT[x] += 1
        if len(TO_LCM) == len(WATCH):
            print(lcm(TO_LCM))
            sys.exit(0)

        if x == 'rx' and typ == 'lo':
            print(t + 1)

        if typ == 'lo':
            lo += 1
        else:
            hi += 1

        if x not in R:
            continue
        if x == 'broadcaster':
            for y in R[x]:
                Q.append((y, x, typ))
        elif x[0] == '%':
            if typ == 'hi':
                continue
            else:
                if x not in ON:
                    ON.add(x)
                    new_typ = 'hi'
                else:
                    ON.discard(x)
                    new_typ = 'lo'
                for y in R[x]:
                    Q.append((y, x, new_typ))
        elif x[0] == '&':
            FROM[x][from_] = typ
            new_typ = ('lo' if all(y == 'hi' for y in FROM[x].values()) else 'hi')
            for y in R[x]:
                Q.append((y, x, new_typ))
        else:
            assert False, x
    if t == 1000:
        print(lo * hi)


681194780
238593356738827


NameError: ignored